In [1]:
import gym
import keras_gym as km
from tensorflow.keras.layers import Conv2D, Lambda, Dense, Flatten
from tensorflow.keras import backend as K

/home/vinicius/.local/lib/python3.8/site-packages/keras_gym/base/patches.py:23: UserWarning: keras-gym has known issues with eager execution mode; eager execution has been disabled as a precaution. You may try and enable eager execution with: tf.compat.v1.enable_eager_execution()
  warnings.warn(


In [2]:
env = gym.make('Freeway-v0')
env = km.wrappers.ImagePreprocessor(env, height=84, width=84, grayscale=True)
env = km.wrappers.FrameStacker(env, num_frames=3)
env = km.wrappers.TrainMonitor(env)

In [3]:
km.enable_logging()

In [4]:
class Func(km.FunctionApproximator):
    def body(self, S):
        def diff_transform(S):
            S = K.cast(S, 'float32') / 255
            M = km.utils.diff_transform_matrix(num_frames=3)
            return K.dot(S, M)

        X = Lambda(diff_transform)(S)
        X = Conv2D(filters=32, kernel_size=8, strides=4, activation='relu')(X)
        #X = Conv2D(filters=64, kernel_size=4, strides=2, activation='relu')(X)
        #X = Conv2D(filters=64, kernel_size=3, strides=2, activation='relu')(X)
        X = Flatten()(X)
        #X = Dense(units=512, activation='relu')(X)
        X = Dense(units=256, activation='relu')(X)
        return X

In [5]:
func = Func(env, lr=0.5)
pi = km.SoftmaxPolicy(func, update_strategy='ppo')
v = km.V(func, gamma=0.99, bootstrap_n=10, bootstrap_with_target_model=True)
actor_critic = km.ActorCritic(pi, v)

In [6]:
buffer = km.caching.ExperienceReplayBuffer.from_value_function(
    value_function=v, capacity=1000, batch_size=64)

In [9]:
while env.T < 10000:
    s = env.reset()

    for t in range(env.spec.max_episode_steps):
        a = pi(s, use_target_model=True)  # target_model == pi_old
        s_next, r, done, info = env.step(a)

        buffer.add(s, a, r, done, env.ep)

        if len(buffer) >= buffer.capacity:
            num_batches = int(4 * buffer.capacity / buffer.batch_size)
            for _ in range(num_batches):
                actor_critic.batch_update(*buffer.sample())
            buffer.clear()

            actor_critic.sync_target_model(tau=0.1)

        if done:
            break

        s = s_next

    if env.ep % 2 == 0:
        km.utils.generate_gif(
            env=env,
            policy=pi,
            filepath='Logs/{:06d}.gif'.format(env.ep),
            resize_to=(320, 420))

INFO:TrainMonitor:ep: 3, T: 4,754, G: 0, avg_G: 0, t: 2731, dt: 17.242ms, loss: -0.011, policy/entropy: 1.1, policy/kl_div: 0, policy/loss: -0.011, value/loss: 0
INFO:TrainMonitor:ep: 4, T: 7,487, G: 0, avg_G: 0, t: 2732, dt: 16.445ms, loss: -0.011, policy/entropy: 1.1, policy/kl_div: 0, policy/loss: -0.011, value/loss: 0
INFO:TrainMonitor:ep: 5, T: 10,225, G: 0, avg_G: 0, t: 2737, dt: 10.874ms, loss: -0.011, policy/entropy: 1.1, policy/kl_div: 0, policy/loss: -0.011, value/loss: 0
INFO:generate_gif:recorded episode to: Logs/000004.gif
